## Team Challenge Part 2: Automated landmark placement

In [ ]:
"""
Code is provided by Adaloglou Nikolas (2021)

https://github.com/black0017/ct-intensity-segmentation

"""
import os
import csv
import cv2
import glob
import math
import time
import shutil

import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

import scipy.ndimage as ndimage
from PIL import Image, ImageDraw
from scipy.spatial import ConvexHull
from matplotlib.backend_bases import MouseButton
from skimage.segmentation import flood, flood_fill
from skimage import measure, filters, color, morphology
from skimage.morphology import erosion, dilation, area_closing, square, remove_small_objects, h_maxima

from ScrollPlot import ScrollPlot



def NormalizeData(data):
    """ Normalizes data to the [0, 255] range; not required right now """
    return (data - np.min(data)) / (np.max(data) - np.min(data)) * 255

def visuals(slice_nr):
    ### VISUALIZATION
    #Regular image
    plt.imshow(img_data[:,:,slice_nr],cmap='gray')
    plt.show()

    # Thresholded image
    plt.imshow(thresholded_3d[:,:,slice_nr],cmap='gray')
    plt.show()

    # Image with holes filled and spine isolated
    plt.imshow(dl_image_3d[:,:,slice_nr],cmap='gray')
    plt.show()

    plt.figure()
    plt.imshow(img_data[:,:,slice_nr], cmap='gray')
    plt.imshow(dl_image_3d[:,:,slice_nr], cmap='jet', alpha=0.4)
    plt.show()


In [ ]:
def segmentation_data(img_data,file):

    # Start timer
    start_time = time.time()

    # Set parameters
    MIN_THRESHOLD_BONE = 210
    MAX_THRESHOLD_ANTI_METAL = 800 # for post-op images only
    MIN_THRESHOLD_STRUCTURE_SIZE = 1500

    # Screw reduction (for post-op images only)
    if file[1:6] == 'preop' or file[1:7] == 'postop':
        img_data[img_data > MAX_THRESHOLD_ANTI_METAL] = MAX_THRESHOLD_ANTI_METAL

    # Gaussian blurring
    img_data_blurred = ndimage.gaussian_filter(img_data, sigma=(3, 3, 3), order=0)

    # Create empty arrays to store data in
    thresholded_3d = np.zeros(img_data.shape)
    dl_image_3d = np.zeros(img_data.shape)

    for slice in range(img_data.shape[2]):
        # Slice selection
        image = img_data_blurred[:,:,slice]

        # Thresholding
        th_image = image > MIN_THRESHOLD_BONE

        # Spine isolation
        dl_image = ndimage.morphology.binary_fill_holes(th_image)
        dl_image = remove_small_objects(dl_image, min_size=MIN_THRESHOLD_STRUCTURE_SIZE)

        # Store 3D data
        thresholded_3d[:,:,slice] = th_image
        dl_image_3d[:,:,slice] = dl_image

    print("Processing the image took %s seconds." % (time.time() - start_time))
    #print("To visualize a single slice, use 'python -i LoadP4.3D' and type 'visuals(slice_nr)' when the processing is done!")
    
    return img_data, dl_image_3d

def get_manual_landmarks(img_title, img_data, dl_image_3d):
    
    %matplotlib qt

    fig, ax = plt.subplots(1, 1)
    sp = ScrollPlot(img_title, ax, img_data, dl_image_3d)
    fig.canvas.mpl_connect('scroll_event', sp.on_scroll)
    fig.canvas.mpl_connect('button_press_event', sp.on_click)
    plt.show()

    plt.waitforbuttonpress()
    plt.close()
    
    return sp.get_landmark()

# Manual landmark placement

In [ ]:
# Load the image data
path = os.path.abspath('')
preop_file = '/Scoliose/1preop.nii'
postop_file = '/Scoliose/1postop.nii'

preop_img = nib.load(path+preop_file)
preop_img_data = preop_img.get_fdata()

postop_img = nib.load(path+postop_file)
postop_img_data = postop_img.get_fdata()

# Display the scrollplot for 1 manual landmark per dataset
preop_data, dl_preop_3d = segmentation_data(preop_img_data,preop_file)
postop_data, dl_postop_3d = segmentation_data(postop_img_data,postop_file)

preop_lm = get_manual_landmarks('Preop image', preop_data, dl_preop_3d)
postop_lm = get_manual_landmarks('Postop image', postop_data, dl_postop_3d)

print('Landmark location preop: ',preop_lm)   # (x,y,slice)
print('Landmark location postop: ',postop_lm)

# Image translation

In [ ]:
# It translates each datapoint of the postop image and fills in the gaps with the minimum value

%matplotlib inline 
t_data = postop_data
min_value = np.amin(postop_data)

# Get differences of each axis
x_diff = preop_lm[0] - postop_lm[0]
y_diff = preop_lm[1] - postop_lm[1]
slice_diff = preop_lm[2] - postop_lm[2]

# Slice translation
t_data = np.roll(t_data, slice_diff, axis=2)
if slice_diff < 0:
    t_data[:,slice_diff:] = min_value
else:
    t_data[:,:slice_diff] = min_value
        
# Row and column translation
for slice in range(t_data.shape[2]):
    t_data[:,:,slice] = np.roll(t_data[:,:,slice], x_diff, axis=0)
    t_data[:,:,slice] = np.roll(t_data[:,:,slice], y_diff, axis=1)
    
    if x_diff < 0:
        t_data[x_diff:,:,slice] = min_value
    else:
        t_data[:x_diff,:,slice] = min_value

    if y_diff < 0:
        t_data[:,y_diff:,slice] = min_value
    else:
        t_data[:,:y_diff,slice] = min_value

fig, ax = plt.subplots(1, 2, figsize=(20, 5))
ax[0].imshow(postop_data[:,:,150], cmap='gray')
ax[0].set_title('Original postop image')
ax[1].imshow(t_data[:,:,150], cmap='gray')
ax[1].set_title('Translated postop image')